#### | AutoML Using PSO-SVM|

In [ ]:
import pandas as pd
import numpy as np
from sklearn.metrics import mean_absolute_error,r2_score,mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn import svm
from sklearn.metrics import mean_squared_error

In [ ]:
#import the dataset 
test = pd.read_excel('https://github.com/AufaAnasin/bostonHousingNA/blob/main/data_set/data_set/boston_test.xlsx?raw=true')
train = pd.read_excel('https://github.com/AufaAnasin/bostonHousingNA/blob/main/data_set/data_set/boston_train.xlsx?raw=true')
# assign into feature and target variable
X_train = train.iloc[:,:-1]
y_train = train.iloc[:, [-1]]

X_test = test.iloc[:,:-1]
y_test = test.iloc[:, [-1]]

# split the training data into train and valid dataset
X_train, X_valid, y_train, y_valid = train_test_split(X_train, y_train, random_state=0, train_size = .7)
X_train.shape, X_valid.shape

((282, 13), (122, 13))

In [ ]:
X_train.head(5)

,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT
327,0.04741,0.0,11.93,0,0.573,6.030,80.8,2.5050,1,273,21.0,396.90,7.88
221,4.34879,0.0,18.10,0,0.580,6.167,84.0,3.0334,24,666,20.2,396.90,16.29
354,0.13117,0.0,8.56,0,0.520,6.127,85.2,2.1224,5,384,20.9,387.69,14.09
49,0.03306,0.0,5.19,0,0.515,6.059,37.3,4.8122,5,224,20.2,396.14,8.51
368,12.04820,0.0,18.10,0,0.614,5.648,87.6,1.9512,24,666,20.2,291.55,14.10


### Hyper Parameter Tuning 

In [ ]:
# defining parameter range
param_grid = {'C': [0.1, 1, 10, 100, 1000],
              'kernel': ['rbf', 'poly', 'sigmoid'],
#               'kernel': ['rbf','linear', 'poly'],
              'epsilon':[0.001, 0.01, 0.1, 1, 10, 100, 1000],
              'degree':[1, 2, 3, 4, 5, 6],
#               'gamma' :['auto','scale']
             }
 
grid = GridSearchCV(svm.SVR(), param_grid, refit = True, verbose = 3)
 
# fitting the model for grid search
grid.fit(X_train, y_train.values.ravel())

Fitting 5 folds for each of 630 candidates, totalling 3150 fits
[CV 1/5] END C=0.1, degree=1, epsilon=0.001, kernel=rbf;, score=0.002 total time=   0.0s
[CV 2/5] END C=0.1, degree=1, epsilon=0.001, kernel=rbf;, score=0.136 total time=   0.0s
[CV 3/5] END C=0.1, degree=1, epsilon=0.001, kernel=rbf;, score=0.121 total time=   0.0s
[CV 4/5] END C=0.1, degree=1, epsilon=0.001, kernel=rbf;, score=0.015 total time=   0.0s
[CV 5/5] END C=0.1, degree=1, epsilon=0.001, kernel=rbf;, score=0.106 total time=   0.0s
[CV 1/5] END C=0.1, degree=1, epsilon=0.001, kernel=poly;, score=-0.015 total time=   0.0s
[CV 2/5] END C=0.1, degree=1, epsilon=0.001, kernel=poly;, score=0.110 total time=   0.0s
[CV 3/5] END C=0.1, degree=1, epsilon=0.001, kernel=poly;, score=0.105 total time=   0.0s
[CV 4/5] END C=0.1, degree=1, epsilon=0.001, kernel=poly;, score=0.010 total time=   0.0s
[CV 5/5] END C=0.1, degree=1, epsilon=0.001, kernel=poly;, score=0.083 total time=   0.0s
[CV 1/5] END C=0.1, degree=1, epsilon=0.

GridSearchCV(estimator=SVR(),
             param_grid={'C': [0.1, 1, 10, 100, 1000],
                         'degree': [1, 2, 3, 4, 5, 6],
                         'epsilon': [0.001, 0.01, 0.1, 1, 10, 100, 1000],
                         'kernel': ['rbf', 'poly', 'sigmoid']},
             verbose=3)

In [ ]:
selected_kernel = grid.best_params_['kernel']
selected_C = grid.best_params_['C']
# selected_gamma = grid.best_params_['gamma']
selected_epsilon = grid.best_params_['epsilon']
selected_degree = grid.best_params_['degree']
# print best parameter after tuning
print("Selected Kernel: ", selected_kernel)
print("Selected C: ", selected_C)
# print("Selected Gamma: ", selected_gamma)
print("Selected Epsilon: ", selected_epsilon)
print("Selected Degree: ", selected_degree)

 
# print how our model looks after hyper-parameter tuning
print("Model Looks: ", grid.best_estimator_)

Selected Kernel:  rbf
Selected C:  1000
Selected Epsilon:  1
Selected Degree:  1
Model Looks:  SVR(C=1000, degree=1, epsilon=1)


#### Model

In [ ]:
# define your own objective function, make sure the function receives four parameters,
#  fit your model and return the objective value !
def objective_function_topass(model, X_train, y_train, X_valid, y_valid):      
    model.fit(X_train.values,y_train)  
    P=mean_squared_error(y_valid,model.predict(X_valid.values))
    return P
# import an algorithm !  
from zoofs import ParticleSwarmOptimization
# create object of algorithm
algo_object=ParticleSwarmOptimization(objective_function_topass,n_iteration=20,
                                       population_size=20,minimize=True)

#Modelling using SVR
SVR_model = grid.best_estimator_                                       
# fit the algorithm
algo_object.fit(SVR_model, X_train, y_train.values.ravel(), X_valid, y_valid,verbose=True)
#plot your results
algo_object.plot_history()

 [ 2022-06-26 12:53:09,712 ] Finished iteration #0 with objective value 24.01566719467559. Current best value is 24.01566719467559 
 [ 2022-06-26 12:53:10,070 ] Finished iteration #1 with objective value 16.249648435727615. Current best value is 16.249648435727615 
 [ 2022-06-26 12:53:10,456 ] Finished iteration #2 with objective value 16.674748068289787. Current best value is 16.249648435727615 
 [ 2022-06-26 12:53:10,817 ] Finished iteration #3 with objective value 20.47597134955811. Current best value is 16.249648435727615 
 [ 2022-06-26 12:53:11,187 ] Finished iteration #4 with objective value 19.91956822273714. Current best value is 16.249648435727615 
 [ 2022-06-26 12:53:11,681 ] Finished iteration #5 with objective value 13.1790490630693. Current best value is 13.1790490630693 
 [ 2022-06-26 12:53:12,084 ] Finished iteration #6 with objective value 16.249648435727615. Current best value is 13.1790490630693 
 [ 2022-06-26 12:53:12,388 ] Finished iteration #7 with objective value 

In [ ]:
def run_experiment(SVR_model):
    SVR_model.fit(X_train, y_train.values.ravel())
    y_pred = SVR_model.predict(X_test)
    print("R^2 : ", r2_score(y_test, y_pred))
    print("MAE :", mean_absolute_error(y_test,y_pred))
    print("RMSE:",np.sqrt(mean_squared_error(y_test, y_pred)))
    
# Model Evaluation
run_experiment(SVR_model)

R^2 :  0.5992771329580644
MAE : 4.1128833259647575
RMSE: 6.4135586022493065
